# 📘 Final Project: Data Analysis using MongoDB and Apache Spark

This project demonstrates how to work with a real-world dataset (Amazon Books Reviews Dataset) using MongoDB for storage and Apache Spark (PySpark) for processing. We explore schema design, querying, performance optimization, and visual insights.

---

# Step 1: Import libraries
import pandas as pd
import numpy as np
import json
from pymongo import MongoClient

In [ ]:
# Step 2: Configure Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Step 3: Load CSV files
books_df = pd.read_csv("books_data.csv")
ratings_df = pd.read_csv("Books_rating.csv")

## 📂 Dataset Overview

We use the **Amazon Books Reviews** dataset from Kaggle, which contains user-generated reviews for a wide variety of books available on Amazon. This real-world dataset includes valuable attributes such as:

- **Title**: The name of the book.
- **Author(s)**: The author(s) of the book.
- **Categories**: Genre or subject of the book (e.g., Comics, Fiction, Education).
- **Rating**: User rating scores, typically from 1 to 5.
- **Review Text**: Actual written review provided by users.
- **Review Date**: When the review was posted.
- **ASIN**: Unique Amazon product identifier for each book.

This dataset provides an excellent foundation for exploring data storage, processing, and analysis using MongoDB and Apache Spark due to its unstructured nature and scale.

In [ ]:
# Step 4: Basic exploration
print("📘 Books Data Sample:")
print(books_df.head(), "\n")

print("📝 Ratings Data Sample:")
print(ratings_df.head(), "\n")

print("📊 Shapes:")
print("Books Data Shape:", books_df.shape)
print("Ratings Data Shape:", ratings_df.shape, "\n")

print("🔍 Missing Values in Books Data:")
print(books_df.isnull().sum(), "\n")

print("🔍 Missing Values in Ratings Data:")
print(ratings_df.isnull().sum(), "\n")

In [ ]:
# Step 5: Optional Cleaning
# Drop rows with missing essential review text
ratings_df = ratings_df.dropna(subset=["review/text"])


# Fill missing summaries with a placeholder (safe version)
ratings_df["review/summary"] = ratings_df["review/summary"].fillna("No summary provided")


# You can apply similar cleaning to books_df if needed
# books_df = books_df.dropna()  # example